In [1]:
import os,shutil
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import zipfile
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import sklearn



In [4]:
print("Tis is test github")

Tis is test github


In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [3]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.express as px
from plotly.subplots import make_subplots
print("Tensorflow Version: {}".format(tf.__version__))

Tensorflow Version: 2.5.0


In [ ]:
# Data Import
import kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Kaggle API to download the dataset in local drive
#! kaggle competitions download -c plant-pathology-2020-fgvc7

root = os.getcwd()

zipFile = 'plant-pathology-2020-fgvc7.zip'
root = os.getcwd()
filePath = os.path.join(root,zipFile)

print("[INFO] Extracting downloaded zip dataset...... \n")
if os.path.exists(os.path.join(root,'images')) == False:
    with zipfile.ZipFile(filePath, 'r') as file:
        file.extractall()      
print("[INFO] Zip extract completed...... \n")


print("[INFO] Generating train dataframe...... \n")

trainDf = pd.read_csv(os.path.join(root, "train.csv"))
print("[INFO] Total shape of train dataset: {}".format(trainDf.shape))

  
# function to get column name with value one for each row in dataframe
def get_class(row):
    return(row.index[row.apply(lambda x: x==1)][0])

# prepare a ClassName column
trainDf['DiseaseClass'] = trainDf.apply(lambda row:get_class(row), axis=1)

trainDf.tail(8)

In [ ]:
# split train dataset into train and validation (80-20 ratio)
from sklearn.model_selection import train_test_split
y = trainDf["DiseaseClass"]
df_train, df_val = train_test_split(trainDf, test_size=0.2, random_state=42, stratify=y)

print("[INFO] Shape of train dataset is: {}\n".format(df_train.shape))
print("[INFO] Shape of validation dataset is: {}\n".format(df_val.shape))

In [ ]:
trainDir = os.path.join(root, 'train') # train directory
valDir = os.path.join(root, "val")     # validation directory
testDir = os.path.join(root, 'test')   #test directory


print("[INFO] Creating separate dirs for disease class in train directory ! \n")
if os.path.exists(trainDir) == False:  # if directory is absent craete one
    os.mkdir('train')
    trainDir = os.path.join(root, 'train') # train directory
    os.mkdir(os.path.join(trainDir, "healthy"))
    os.mkdir(os.path.join(trainDir, "rust"))
    os.mkdir(os.path.join(trainDir, "scab"))
    os.mkdir(os.path.join(trainDir, "multiple_diseases"))
    trainImage = list(df_train["image_id"])
#     for filename in os.listdir(os.path.join(root,"images")):
#         tail = os.path.split(filename)[-1]
#         if tail.split("_")[0]== "Train":
#             trainImage.append(filename)
            
    for file in trainImage:
        file = file + '.jpg'
        index = trainDf.loc[trainDf["image_id"] == file.split(".")[0], "DiseaseClass"].index[0]
        label = trainDf.loc[trainDf["image_id"] == file.split(".")[0], "DiseaseClass"][index]
        src = os.path.join(root, 'images',file)
        dst = os.path.join(trainDir, label, file )
        shutil.copy2(src, dst)
        
print("[INFO] Creating separate dirs for disease class in validation directory ! \n") 
if os.path.exists(valDir) == False:  # if directory is absent craete one
    os.mkdir('val')
    valDir = os.path.join(root, "val") # validation directory
    os.mkdir(os.path.join(valDir, "healthy"))
    os.mkdir(os.path.join(valDir, "rust"))
    os.mkdir(os.path.join(valDir, "scab"))
    os.mkdir(os.path.join(valDir, "multiple_diseases"))
    
    valImage = list(df_val["image_id"])
    for file in valImage:
        file = file + '.jpg'
        index = trainDf.loc[trainDf["image_id"] == file.split(".")[0], "DiseaseClass"].index[0]
        label = trainDf.loc[trainDf["image_id"] == file.split(".")[0], "DiseaseClass"][index]
        src = os.path.join(root, 'images',file)
        dst = os.path.join(valDir, label, file )
        shutil.copy2(src, dst)
        

print("[INFO] Creating separate test directory ! \n")
if os.path.exists(testDir) == False:  # if directory is absent craete one
    os.mkdir('test')
    testDir = os.path.join(root, 'test')
    testImage = []
    for filename in os.listdir(os.path.join(root,"images")):
        tail = os.path.split(filename)[-1]
        if tail.split("_")[0]== "Test":
            testImage.append(filename)
    for file in testImage:
        src = os.path.join(os.path.join(root, 'images'),file)
        dst = os.path.join(root, 'test')
        shutil.copy2(src, dst)


print("[INFO] Total test images in test directory: {}".format(len(os.listdir(testDir))))


## TARGET CLASS

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.parallel_categories(trainDf, dimensions=[ 'healthy', 'rust', 'multiple_diseases', 'scab'],
                             color = "healthy",
                color_continuous_scale=px.colors.sequential.Inferno,
                labels={'rust':'RUST LEAF', 'healthy':'HEALTHY LEAF', 'multiple_diseases':'MULTIPLE DISEASES', 'scab':'SCAB'},
                            title="PARALLEL CATEGORICAL PLOT OF DISEASE CLASS ")
fig.show()

In [ ]:
className = ["healthy", "multiple_diseases", "rust", "scab"]
classCount = []
for name in className:
    result = sum(trainDf[trainDf[name]==1].value_counts())
    classCount.append(result)
    
data = {"ClassName":className, "Count":classCount}  
countDf = pd.DataFrame(data=data)

fig = px.bar(countDf, x='ClassName',y='Count', title="Distribution of Image Disease Class in Train Dataset",
            width=800, height=400)

fig.update_traces(marker_color='rgb(150,100,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.4)
fig.show()

In [ ]:
# Dataset Visulization
images = os.listdir(os.path.join(root,"images"))[:12]
images
className = []    
plt.figure(figsize = (12,12))
for i,img in enumerate(images):
    imgPath = os.path.join(root,"images",img)
    plt.subplot(4,3,i+1)
    plt.axis('off')
    #plt.title(title, size=9, color='black')
    img = cv2.imread(imgPath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    #fig = px.imshow(img)
    #fig.show()
plt.show()

In [ ]:
# Visulaization of a single leaf

image = os.listdir(os.path.join(root, "train", "rust"))[23]
index = trainDf.loc[trainDf["image_id"]==image.split(".")[0], "DiseaseClass"].index[0]
title = trainDf.loc[trainDf["image_id"]==image.split(".")[0], "DiseaseClass"][index]
imgPath = os.path.join(root, "train", "rust", image)
img = cv2.imread(imgPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   #for cv2 conversion from BGR to RGB
fig = px.imshow(img)
fig.update_layout(title_text=str(title), title_x=0.5)
fig.show()


In [ ]:
file = os.listdir(os.path.join(root,"train","multiple_diseases"))[9]
imgPath = os.path.join(root, "train", "multiple_diseases", file)
img = cv2.imread(imgPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = make_subplots(1,2)
fig.add_trace(go.Image(z=img), 1, 1)
for channel,color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[...,channel].ravel(), opacity = 0.5,
                              marker_color = color, name = '%s Channel'%color), 1, 2)
fig.update_layout(height = 400, title = "LEAF WITH MULTIPLE DISEASE", title_x=0.5)
fig.show()

In [ ]:
file = os.listdir(os.path.join(root,"train","healthy"))[9]
imgPath = os.path.join(root, "train", "healthy", file)

img = cv2.imread(imgPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = make_subplots(1,2)
fig.add_trace(go.Image(z=img), 1, 1)
for channel,color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[...,channel].ravel(), opacity = 0.5,
                              marker_color = color, name = '%s Channel'%color), 1, 2)
fig.update_layout(height = 400, title = "HEALTHY LEAF", title_x=0.5)
fig.show()

In [ ]:
file = os.listdir(os.path.join(root,"train","rust"))[9]
imgPath = os.path.join(root, "train", "rust", file)

img = cv2.imread(imgPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = make_subplots(1,2)
fig.add_trace(go.Image(z=img), 1, 1)
for channel,color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[...,channel].ravel(), opacity = 0.5,
                              marker_color = color, name = '%s Channel'%color), 1, 2)
fig.update_layout(height = 400, title = "RUST LEAF", title_x=0.5)
fig.show()

In [ ]:
file = os.listdir(os.path.join(root,"train","scab"))[9]
imgPath = os.path.join(root, "train", "scab", file)

img = cv2.imread(imgPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = make_subplots(1,2)
fig.add_trace(go.Image(z=img), 1, 1)
for channel,color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[...,channel].ravel(), opacity = 0.5,
                              marker_color = color, name = '%s Channel'%color), 1, 2)
fig.update_layout(height = 400, title = "SCAB LEAF", title_x=0.5)
fig.show()

# IMAGE PRE-PROCESSING AND IMAGE AUGMENTATION

## CANNY EDGE DETECTION

For detecting strong edge of the image, reducing noise in the image by using Gaussian kernel

In [ ]:
img = os.listdir(trainDir)[:50]
plt.figure(figsize = (12,12))

imgPath = os.path.join(trainDir, img[10])

image = cv2.imread(imgPath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# CANNY EDGE ALGORITHM
edges = cv2.Canny(image, 100, 200)

# VISUALIZE THE RESULT
plt.figure(figsize=(8,5))
plt.subplot(121)
plt.axis('off')
plt.title('ORIGINAL')
plt.imshow(image[:,:,:])
                 
plt.subplot(122)
plt.axis('off')
plt.title('EDGES')
plt.imshow(edges, cmap='gray')
plt.tight_layout()
plt.show()
